# RAG chatbot application using LangChain RetrievalQA Chain and Chainlit with Llama-2 7b gptq model and expose web server by ngrok



In [ ]:
!pip install langchain chainlit pyngrok google-search-results transformers==4.37.2 pypdf sentence-transformers chromadb auto-gptq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 814.5/814.5 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 15.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 64.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 79.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.4/278.4 kB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.2/104.2 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.

In [ ]:
import langchain
import chainlit
import pyngrok
import transformers
import pypdf
import chromadb
import auto_gptq


print(langchain.__version__)
print(chainlit.__version__)
print(pyngrok.__version__)
print(transformers.__version__)
print(pypdf.__version__)
print(chromadb.__version__)
print(auto_gptq.__version__)

0.1.15
1.0.502
7.1.6
4.37.2
4.2.0
0.4.24
0.7.1


In [ ]:
from langchain.document_loaders import PyPDFLoader
from getpass import getpass
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA, LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.memory import ConversationBufferMemory
from langchain import OpenAI
from langchain.embeddings import HuggingFaceEmbeddings
import torch
from langchain import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline, TextStreamer
from auto_gptq import AutoGPTQForCausalLM
from langchain.agents import AgentOutputParser
from langchain.agents.conversational_chat.prompt import FORMAT_INSTRUCTIONS
from langchain.output_parsers.json import parse_json_markdown
from langchain.schema import AgentAction, AgentFinish
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chains import ConversationalRetrievalChain
import os

In [ ]:
if not os.path.exists("Vector_db"):
  os.makedirs("Vector_db")

loader = PyPDFLoader("/content/e_budget_speech_2022-23.pdf")
data = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(data)

embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
Vector_db_01 = Chroma.from_documents(docs, embeddings, collection_name="23budget", persist_directory="Vector_db")
Vector_db_01.persist()
Vector_db_01 = None

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
Vector_db_01 = Chroma(persist_directory="Vector_db", collection_name="23budget", embedding_function=embeddings)
Vector_db_01.get()
Vector_db_01.similarity_search("Is there any economic support for start up company in Hong Kong 2023 budget?")

[Document(page_content='Enhancing Economic Resilience and Enriching Industrial Development  \n \n \n23 55. Over the past few years, apart from the $2  billion \nInnovation and Technology Venture Fund, we have also set up the \nCorporate Venture Fund and the Cyberport Macro Fund through the \nHKSTPC and the Cyberport respectively.  Each of these funds has its own specific key areas of investment, which has significantly broadened the fundraising channel for start -ups in Hong Kong.  \n56. Nevertheless, as the funds mentioned above are mainly \ntargeted at early- stage start -ups, they may not be suitable for expanding \nenterprises with considerable scale.  Some such type of enterprises \noften have huge development potential.  Indeed, we are not short of \nexamples of success in nurturing these enterprises in Hong Kong.  \n57. I announced in my Budget two years ago the setting up of \nan investment portfolio named the Hong Kong Growth Portfolio using part of the Future Fund for investm

In [ ]:
%%writefile app.py
from langchain.document_loaders import PyPDFLoader
from getpass import getpass
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA, ConversationalRetrievalChain, LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.memory import ConversationBufferMemory
from langchain import OpenAI
from langchain.embeddings import HuggingFaceEmbeddings
import torch
from langchain import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline, TextStreamer
from auto_gptq import AutoGPTQForCausalLM
from langchain.agents import AgentOutputParser
from langchain.agents.conversational_chat.prompt import FORMAT_INSTRUCTIONS
from langchain.output_parsers.json import parse_json_markdown
from langchain.schema import AgentAction, AgentFinish
from langchain.prompts import PromptTemplate
import os
import chainlit as cl

def set_prompt():

  pre_prompt ="""[INST] <<SYS>>
  You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.

  If you don't know the answer, just say that you don't know, don't try to make up an answer.
  <</SYS>>

  Use the following pieces of information to answer the user's question. If you cannot answer the question from the given documents, please state that you do not have an answer.\n"""

  prompt = pre_prompt + "CONTEXT:\n\n{context}\n" +"Question : {question}" + "[\INST]"
  llama_prompt = PromptTemplate(template=prompt, input_variables=["context", "question"])

  return llama_prompt

def load_llm():

  model_name = "TheBloke/Llama-2-7B-chat-GPTQ"
  # model_name = "TheBloke/Llama-2-13B-chat-GPTQ"
  model_basename = "model"
  revision="gptq-4bit-128g-actorder_True"
  DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

  tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

  model = AutoGPTQForCausalLM.from_quantized(
      model_name,
      revision=revision,
      model_basename=model_basename,
      use_safetensors=True,
      trust_remote_code=True,
      inject_fused_attention=False,
      device=DEVICE,
      quantize_config=None,)

  streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

  text_pipeline = pipeline(
      "text-generation",
      model=model,
      tokenizer=tokenizer,
      do_sample=False,
      # temperature=0,
      max_new_tokens=1024,
      streamer=streamer,
  )

  llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0.1})

  return llm

def qa_bot():

  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
  Vector_db_01 = Chroma(persist_directory="Vector_db", collection_name="23budget", embedding_function=embeddings)
  Vector_db_01.get()

  llm = load_llm()
  qa_chain_prompt = set_prompt()

  qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type='stuff', retriever=Vector_db_01.as_retriever(),
                      return_source_documents=True, chain_type_kwargs={'prompt': qa_chain_prompt})

  return qa_chain

@cl.on_chat_start
async def start():
  chain = qa_bot()
  # msg = cl.Message(content="The bot is loading...")
  # await msg.send()
  # msg.content = "Hi, Welcome to Hong Kong 2022-2023 Budget QA Bot. What is your query?"
  # await msg.update()

  cl.user_session.set("chain", chain)

  msg = cl.Message(content="Hi, Welcome to Hong Kong 2022-2023 Budget QA Bot. What is your query?")
  await msg.send()

@cl.on_message
async def main(message: cl.Message):
    chain = cl.user_session.get("chain")
    cb = cl.AsyncLangchainCallbackHandler(
        stream_final_answer=True, answer_prefix_tokens=["FINAL", "ANSWER"]
    )
    cb.answer_reached = True
    res = await chain.acall(message.content, callbacks=[cb])
    answer = res["result"][res["result"].find("[\\INST]") + len("[\\INST]"):].strip()

    # sources = res["source_documents"]

    # if sources:
    #     answer += f"\nSources:" + str(sources)
    # else:
    #     answer += "\nNo sources found"

    await cl.Message(content=answer).send()

Writing app.py


In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!chainlit run app.py &>/content/logs.txt &

In [ ]:
!ngrok config add-authtoken 2dwhwTxINozPijfRoIeQJOVmbcW_3MBz42EEYpDrS2efbsXUe

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from pyngrok import ngrok
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)

Public URL: https://937f-34-90-231-71.ngrok-free.app


In [ ]:
ngrok.disconnect(8000)
ngrok.kill()